# **<font color = '#3498eb'>SECTOR & COMPANY</font>**

Extreme Sports House is a mountain and adventure distribution company. 

The outdoor apparel and gear sector, which encompasses mountain and adventure clothing and objects, has seen significant growth over the years. According to reports by the Outdoor Industry Association, consumer spending on outdoor recreation totaled $887 billion in 2017. This growth has been fueled by an increasing consumer focus on health and well-being, as well as a surge in adventure tourism. Various trends, such as the rising popularity of sustainable and eco-friendly products, are shaping the industry.

# **<font color = '#3498eb'>OBJETIVES</font>**

The company has enlisted me to:

· Extract maximum value from their existing data.

· Identify the most profitable channels.

· Determine who their most valuable clients are.

· Develop strategies to increase profit margins.

· Review the product portfolio to cut costs.

· Optimize customer management processes.

· Build a SQL-based recommendation system to increase cross-selling opportunities.

# **<font color = '#3498eb'>LIBRARIES</font>**

In [4]:
import pandas as pd
import pymysql
import tabulate 

# **<font color = '#3498eb'>DATA LOADING AND UNDERSTANDING</font>**

## GENERAL VIEW

In [5]:
eshdb_conn = pymysql.connect(host='localhost', user='root', password='b8SO0ibY5claaghlgiehgld', database='extremesportshouse')

eshdb_cursor = eshdb_conn.cursor()

eshdb_cursor.execute("SELECT DATABASE();")
print("Currently using the database:", eshdb_cursor.fetchone()[0])

Currently using the database: extremesportshouse


I review the tables of the database and then review each table.

In [13]:
eshdb_cursor.execute("show tables from extremesportshouse;")
print(tabulate.tabulate([list(eachtuple) for eachtuple in eshdb_cursor.fetchall()]))

--------
channels
products
sales
stores
--------


In [29]:
eshdb_cursor.execute("select * from channels;")
print(tabulate.tabulate([list(eachtuple) for eachtuple in eshdb_cursor.fetchall()]))

--  -----------
 1  Fax
 2  Telephone
 3  Mail
 4  E-mail
 5  Web
 6  Sales visit
 7  Special
 8  Other
 9  Other
10  Other
11  Other
12  Other
--  -----------


In [19]:
eshdb_cursor.execute("select * from products;")
print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()[:10]], headers=[desc[0] for desc in eshdb_cursor.description]))

  product_id  line               type          product                    brand      color          cost    price
------------  -----------------  ------------  -------------------------  ---------  -----------  ------  -------
        1110  Camping Equipment  Cooking Gear  TrailChef Water Bag        TrailChef  Clear          2.77     6.59
        2110  Camping Equipment  Cooking Gear  TrailChef Canteen          TrailChef  Brown          6.92    12.92
        3110  Camping Equipment  Cooking Gear  TrailChef Kitchen Kit      TrailChef  Unspecified   15.78    23.8
        4110  Camping Equipment  Cooking Gear  TrailChef Cup              TrailChef  Silver         0.85     3.66
        5110  Camping Equipment  Cooking Gear  TrailChef Cook Set         TrailChef  Silver        34.41    54.93
        6110  Camping Equipment  Cooking Gear  TrailChef Deluxe Cook Set  TrailChef  Silver        78.72   129.72
        7110  Camping Equipment  Cooking Gear  TrailChef Single Flame     TrailChef  Silv

In [3]:
eshdb_cursor.execute("select * from sales;")
print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()[:10]], headers=[desc[0] for desc in eshdb_cursor.description]))

  store_id    product_id    channel_id  date          quantity    official_price    offer_price
----------  ------------  ------------  ----------  ----------  ----------------  -------------
      1201        109110             4  12/01/2015         648             76.86          71.48
      1201        112110             4  12/01/2015         799             10.64          10.21
      1201        115110             4  12/01/2015         755             10.71          10.28
      1205         70240             3  12/01/2015          70            122.7          114.11
      1205         71110             3  12/01/2015          28             95.62          92.75
      1215         73110             2  12/01/2015        3992             12.78          11.89
      1215         83110             2  12/01/2015         156             96.44          89.69
      1215         86110             2  12/01/2015        2615              6              5.58
      1215         93110             2  

In [31]:
eshdb_cursor.execute("select * from stores;")
print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()[:10]], headers=[desc[0] for desc in eshdb_cursor.description]))

  store_id  store_name            type                    country
----------  --------------------  ----------------------  -----------
      1101  ActiForme             Equipment Rental Store  France
      1115  SportsClub            Golf Shop               France
      1123  Anapurna              Direct Marketing        France
      1132  Cordages Discount     Warehouse Store         France
      1133  Altitudes extrÃªmes   Outdoors Shop           France
      1134  Optique et Lentilles  Eyewear Store           France
      1135  Camping Sauvage       Outdoors Shop           France
      1137  Grand choix           Department Store        Switzerland
      1144  Die Fitness-Experten  Direct Marketing        Germany
      1147  Der Fitness-Doktor    Sports Store            Germany


## SALES TABLE

The company ask me to complete these tasks:

· Review the types of variables.

· Check if the table is in this level store-product-channel-date.

· Include a new variable with the total revenue of every sale.

In [7]:
eshdb_cursor.execute("describe sales;")
print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()], headers=[desc[0] for desc in eshdb_cursor.description]))

Field           Type    Null    Key    Default    Extra
--------------  ------  ------  -----  ---------  -------
store_id        int     YES     MUL
product_id      int     YES     MUL
channel_id      int     YES     MUL
date            text    YES
quantity        int     YES
official_price  double  YES
offer_price     double  YES


date type should be changed but I'll do it later so I have to create a new table.

Let's check now if the table is in this level: store-product-channel-date. if I do a group by by these variables there shouldn't been duplicated values.

In [17]:
eshdb_cursor.execute("""
    select *, count(*) as repeated_reg
    from sales
    group by store_id, product_id, channel_id, date
    having repeated_reg > 1
    order by store_id, product_id, channel_id, date
    limit 20;
    """)

print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()], headers=[desc[0] for desc in eshdb_cursor.description]))

  store_id    product_id    channel_id  date          quantity    official_price    offer_price    repeated_reg
----------  ------------  ------------  ----------  ----------  ----------------  -------------  --------------
      1115        127110             5  22/12/2016         203             20.15          20.15               2
      1115        127130             5  13/07/2018         133             21.25          21.25               2
      1115        127130             5  22/12/2016         205             20.15          20.15               2
      1115        129130             5  22/12/2016          19            220            220                  2
      1115        130110             5  22/12/2016          24            167.2          167.2                2
      1115        130130             2  16/01/2015           4            172            172                  2
      1115        132120             2  18/10/2015           3             80             80            

Here we can see that there're records that has the same values in store_id, product_id, channel_id and date variables. 

Let's extract the frist example we can see in the above table. 

In [21]:
eshdb_cursor.execute("""
    select *
    from sales
    where store_id=1115 and product_id=127110 and channel_id=5 and date='22/12/2016';
    """)

print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()], headers=[desc[0] for desc in eshdb_cursor.description]))

  store_id    product_id    channel_id  date          quantity    official_price    offer_price
----------  ------------  ------------  ----------  ----------  ----------------  -------------
      1115        127110             5  22/12/2016         203             20.15          20.15
      1115        127110             5  22/12/2016         271             20.15          20.15


We can see here that only quantity value is different.

To have the table in the level required I'll agregate it by store, product, channel and date and I'll apply an aggregation function to the rest of variables, more concretly, the sum in qunatity and the average in official price and offer price. 

As I stated before, I take the opportunity to cast the value of date.

I also will include a new variable with the total revenue of every sale.

In [ ]:
eshdb_cursor.execute("""
    create table agregated_sales as
    select str_to_date(date, '%d/%m/%Y') as order_date, 
    product_id, 
    store_id, 
    channel_id, 
    sum(quantity) as quantity, 
    avg(official_price) as official_price, 
    avg(offer_price) as offer_price, 
    sum(quantity) * avg(offer_price) as total_revenue
    from sales
    group by store_id, product_id, channel_id, order_date
    """)

In [17]:
eshdb_cursor.execute("SELECT * FROM agregated_sales LIMIT 5;")
print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()], headers=[desc[0] for desc in eshdb_cursor.description]))

order_date      product_id    store_id    channel_id    quantity    official_price    offer_price    total_revenue
------------  ------------  ----------  ------------  ----------  ----------------  -------------  ---------------
2015-01-12          109110        1201             4         648             76.86          71.48         46319
2015-01-12          112110        1201             4         799             10.64          10.21          8157.79
2015-01-12          115110        1201             4         755             10.71          10.28          7761.4
2015-01-12           70240        1205             3          70            122.7          114.11          7987.7
2015-01-12           71110        1205             3          28             95.62          92.75          2597


In [20]:
# I execute the code again to see if, effectively, there's no duplicated records now

eshdb_cursor.execute("""
    select *, count(*) as repeated_reg
    from agregated_sales
    group by store_id, product_id, channel_id, order_date
    having repeated_reg > 1;
    """)

print(tabulate.tabulate([list(row) for row in eshdb_cursor.fetchall()], headers=[desc[0] for desc in eshdb_cursor.description]))

order_date    product_id    store_id    channel_id    quantity    official_price    offer_price    total_revenue    repeated_reg
------------  ------------  ----------  ------------  ----------  ----------------  -------------  ---------------  --------------


Now the data science lead in ESH company ask me to modify this new agregated_sales table to relationate it with the rest of tables. Once done, I'll create a new ERModel to document it.

In [7]:
# First I add a primary key to the table and then I modify some others

eshdb_cursor.execute("""
    alter table agregated_sales
        add sales_id int auto_increment primary key,
        add foreign key(product_id) references products(product_id) on delete cascade,
        add foreign key(store_id) references stores(store_id) on delete cascade,
        add foreign key(channel_id) references channels(channel_id) on delete cascade;
    """)

134688

I create the new ER-Model in MysqlWB

<img src="ESH_ermodel.png">

They want to do sometimes analysis by orders (one order has the same date, store and channel) so I'll create a view over the table agregated_sales that icludes it. 